# Lista de Exercícios 08 – Sistemas lineares

## 1 - Para cada um dos sistemas lineares, faça seguinte:
- Reescreva o sistema no formato de matriz estendida;
- Execute a eliminação gaussiana com pivotamento e verifique se o sistema possui solução (o sistema possui solução, se não houverem zeros na diagonal principal após o escalonamento);
- Se o sistema tiver solução, calcule as normas e os números de condicionamento de ordem $1$, $2$ e $\infty$ (no sistema original);
- Se o sistema tiver solução, calcule a mesma usando a fatoração LU (no sistema original).

In [30]:
import numpy as np

def resolve(mat_entrada):
    mat = np.array(mat_entrada, dtype=float)

    A_orig = mat[:, :-1].copy()
    b_orig = mat[:, -1].copy()
    n = len(mat)

    print("-" * 80)
    print("Matriz estendida")
    print("-" * 80)
    print(mat)
    print("-" * 80)
    print("Escalonamento")
    print("-" * 80)

    M = mat.copy()
    P_indices = np.arange(n)

    for c in range(n):
        print(f"Coluna {c+1} :")
        
        idx_maior = np.argmax(np.abs(M[c:, c])) + c
        
        if idx_maior != c:
            print(f"L{idx_maior+1} <-> L{c+1}")
            M[[c, idx_maior]] = M[[idx_maior, c]]
            P_indices[[c, idx_maior]] = P_indices[[idx_maior, c]]
            print(M)
        
        pivo = M[c, c]
        
        if np.isclose(pivo, 0):
            print("Matriz U com zeros na diagonal principal")
            return

        for l in range(c + 1, n):
            elemento = M[l, c]
            if np.isclose(elemento, 0): continue
                
            fator = elemento / pivo
            print(f"L{l+1} <- L{l+1} - {elemento:.4g} / {pivo:.4g} * L{c+1}")
            M[l, :] = M[l, :] - fator * M[c, :]
            M[l, c] = 0.0
            print(M)

    diag = np.diag(M[:, :-1])
    if np.any(np.isclose(diag, 0)):
        print("Matriz U com zeros na diagonal principal")
        return

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        soma = sum(M[i, j] * x[j] for j in range(i + 1, n))
        x[i] = (M[i, n] - soma) / M[i, i]

    print("-" * 80)
    print(f"Solução encontrada (Gauss): \n{x}")

    print("-" * 80)
    print("Normas e Condicionamento (Matriz Original):")
    try:
        normas = {"1": 1, "2": 2, "Inf": np.inf}
        
        for nome, ordem in normas.items():
            norma_val = np.linalg.norm(A_orig, ordem)
            cond_val = np.linalg.cond(A_orig, ordem)
            print(f"Ordem {nome:3}: Norma = {norma_val:8.4f} | Cond = {cond_val:8.4f}")
            
    except np.linalg.LinAlgError:
        print("Matriz singular, impossível calcular condicionamento.")

    print("-" * 80)
    print("Fatoração LU (com pivotamento recuperado do Gauss):")
    A_permutada = A_orig[P_indices]
    b_permutado = b_orig[P_indices]
    
    L = np.eye(n)
    U = A_permutada.copy()
    
    for k in range(n-1):
        for i in range(k+1, n):
            if U[k, k] == 0:
                print("Zero na diagonal durante LU.")
                return
            fator = U[i, k] / U[k, k]
            L[i, k] = fator
            U[i, k:] = U[i, k:] - fator * U[k, k:]
            U[i, k] = 0

    print("Matriz L:\n", L)
    print("Matriz U:\n", U)
    
    y = np.zeros(n)
    for i in range(n):
        soma = sum(L[i, j] * y[j] for j in range(i))
        y[i] = (b_permutado[i] - soma) / L[i, i]
        
    x_lu = np.zeros(n)
    for i in range(n - 1, -1, -1):
        soma = sum(U[i, j] * x_lu[j] for j in range(i + 1, n))
        x_lu[i] = (y[i] - soma) / U[i, i]
        
    print(f"Solução LU: \n{x_lu}")

a) $
\begin{cases}
    2y + 2z = 8 \\
x + 2y +  z = 9 \\
x +  y +  z = 6
\end{cases}
$

In [31]:
mat = [
    [0, 2, 2, 8],
    [1, 2, 1, 9],
    [1, 1, 1, 6]
]
mat = np.array(mat)
resolve(mat)


--------------------------------------------------------------------------------
Matriz estendida
--------------------------------------------------------------------------------
[[0. 2. 2. 8.]
 [1. 2. 1. 9.]
 [1. 1. 1. 6.]]
--------------------------------------------------------------------------------
Escalonamento
--------------------------------------------------------------------------------
Coluna 1 :
L2 <-> L1
[[1. 2. 1. 9.]
 [0. 2. 2. 8.]
 [1. 1. 1. 6.]]
L3 <- L3 - 1 / 1 * L1
[[ 1.  2.  1.  9.]
 [ 0.  2.  2.  8.]
 [ 0. -1.  0. -3.]]
Coluna 2 :
L3 <- L3 - -1 / 2 * L2
[[1. 2. 1. 9.]
 [0. 2. 2. 8.]
 [0. 0. 1. 1.]]
Coluna 3 :
--------------------------------------------------------------------------------
Solução encontrada (Gauss): 
[2. 3. 1.]
--------------------------------------------------------------------------------
Normas e Condicionamento (Matriz Original):
Ordem 1  : Norma =   5.0000 | Cond =  15.0000
Ordem 2  : Norma =   3.9505 | Cond =   8.4591
Ordem Inf: Norma =   4.

b) $\begin{cases}
x + y + z = 0 \\
x +   10z = -48 \\
10y +  z = 25
\end{cases}$

In [32]:
mat_b = [
    [1, 1, 1, 0],
    [1, 0, 10, -48],
    [0, 10, 1, 25]
]
mat_b = np.array(mat_b)
resolve(mat_b)


--------------------------------------------------------------------------------
Matriz estendida
--------------------------------------------------------------------------------
[[  1.   1.   1.   0.]
 [  1.   0.  10. -48.]
 [  0.  10.   1.  25.]]
--------------------------------------------------------------------------------
Escalonamento
--------------------------------------------------------------------------------
Coluna 1 :
L2 <- L2 - 1 / 1 * L1
[[  1.   1.   1.   0.]
 [  0.  -1.   9. -48.]
 [  0.  10.   1.  25.]]
Coluna 2 :
L3 <-> L2
[[  1.   1.   1.   0.]
 [  0.  10.   1.  25.]
 [  0.  -1.   9. -48.]]
L3 <- L3 - -1 / 10 * L2
[[  1.    1.    1.    0. ]
 [  0.   10.    1.   25. ]
 [  0.    0.    9.1 -45.5]]
Coluna 3 :
--------------------------------------------------------------------------------
Solução encontrada (Gauss): 
[ 2.  3. -5.]
--------------------------------------------------------------------------------
Normas e Condicionamento (Matriz Original):
Ordem 1  : Norm

c) $\begin{cases}
-8x + y + z = 1 \\
x -5y + z = 16 \\
x + y - 4z = 7
\end{cases}$

In [33]:
mat_c = [
    [-8, 1, 1, 1],
    [1, -5, 1, 16],
    [1, 1, -4, 7]
]
mat_c = np.array(mat_c)
resolve(mat_c)

--------------------------------------------------------------------------------
Matriz estendida
--------------------------------------------------------------------------------
[[-8.  1.  1.  1.]
 [ 1. -5.  1. 16.]
 [ 1.  1. -4.  7.]]
--------------------------------------------------------------------------------
Escalonamento
--------------------------------------------------------------------------------
Coluna 1 :
L2 <- L2 - 1 / -8 * L1
[[-8.     1.     1.     1.   ]
 [ 0.    -4.875  1.125 16.125]
 [ 1.     1.    -4.     7.   ]]
L3 <- L3 - 1 / -8 * L1
[[-8.     1.     1.     1.   ]
 [ 0.    -4.875  1.125 16.125]
 [ 0.     1.125 -3.875  7.125]]
Coluna 2 :
L3 <- L3 - 1.125 / -4.875 * L2
[[-8.          1.          1.          1.        ]
 [ 0.         -4.875       1.125      16.125     ]
 [ 0.          0.         -3.61538462 10.84615385]]
Coluna 3 :
--------------------------------------------------------------------------------
Solução encontrada (Gauss): 
[-1. -4. -3.]
-----------

d) $\begin{cases}
20x + 7y + 9z = 16 \\
7x +30y + 8z = 38 \\
9x + 8y - 10z = 38
\end{cases}$

In [34]:
mat_d = [
    [20, 7, 9, 16],
    [7, 30, 8, 38],
    [9, 8, -10, 38]
]
mat_d = np.array(mat_d)
resolve(mat_d)

--------------------------------------------------------------------------------
Matriz estendida
--------------------------------------------------------------------------------
[[ 20.   7.   9.  16.]
 [  7.  30.   8.  38.]
 [  9.   8. -10.  38.]]
--------------------------------------------------------------------------------
Escalonamento
--------------------------------------------------------------------------------
Coluna 1 :
L2 <- L2 - 7 / 20 * L1
[[ 20.     7.     9.    16.  ]
 [  0.    27.55   4.85  32.4 ]
 [  9.     8.   -10.    38.  ]]
L3 <- L3 - 9 / 20 * L1
[[ 20.     7.     9.    16.  ]
 [  0.    27.55   4.85  32.4 ]
 [  0.     4.85 -14.05  30.8 ]]
Coluna 2 :
L3 <- L3 - 4.85 / 27.55 * L2
[[ 20.           7.           9.          16.        ]
 [  0.          27.55         4.85        32.4       ]
 [  0.           0.         -14.90381125  25.09618875]]
Coluna 3 :
--------------------------------------------------------------------------------
Solução encontrada (Gauss): 
[ 1

e) $\begin{cases}
9x_1 +  x_2 -2x_3 +  2x_4 = 10.5 \\
 x_1 +15x_2 -3x_3 +  2x_4 = 14.6 \\
 x_1 - 2x_2 +8x_3 +  3x_4 = 18.1 \\
2x_1 + 2x_2 + x_3 + 12x_4 = 19.4
\end{cases}$

In [35]:
mat_e = [
    [9, 1, -2, 2, 10.5],
    [1, 15, -3, 2, 14.6],
    [1, -2, 8, 3, 18.1],
    [2, 2, 1, 12, 19.4]
]
mat_e = np.array(mat_e)
resolve(mat_e)

--------------------------------------------------------------------------------
Matriz estendida
--------------------------------------------------------------------------------
[[ 9.   1.  -2.   2.  10.5]
 [ 1.  15.  -3.   2.  14.6]
 [ 1.  -2.   8.   3.  18.1]
 [ 2.   2.   1.  12.  19.4]]
--------------------------------------------------------------------------------
Escalonamento
--------------------------------------------------------------------------------
Coluna 1 :
L2 <- L2 - 1 / 9 * L1
[[ 9.          1.         -2.          2.         10.5       ]
 [ 0.         14.88888889 -2.77777778  1.77777778 13.43333333]
 [ 1.         -2.          8.          3.         18.1       ]
 [ 2.          2.          1.         12.         19.4       ]]
L3 <- L3 - 1 / 9 * L1
[[ 9.          1.         -2.          2.         10.5       ]
 [ 0.         14.88888889 -2.77777778  1.77777778 13.43333333]
 [ 0.         -2.11111111  8.22222222  2.77777778 16.93333333]
 [ 2.          2.          1.      

f) $\begin{cases}
7x_1 +  x_2 - 2x_3 + 2x_4 = 2.38 \\
 x_1 +14x_2 - 3x_3 + 2x_4 = 5.06 \\
 x_1 - 2x_2 +13x_3 + 3x_4 = 6.88 \\
2x_1 + 2x_2 +  x_3 + 9x_4 = 6.54
\end{cases}$

In [36]:
mat_f = [
    [7, 1, -2, 2, 2.38],
    [1, 14, -3, 2, 5.06],
    [1, -2, 13, 3, 6.88],
    [2, 2, 1, 9, 6.54]
]
mat_f = np.array(mat_f)
resolve(mat_f)

--------------------------------------------------------------------------------
Matriz estendida
--------------------------------------------------------------------------------
[[ 7.    1.   -2.    2.    2.38]
 [ 1.   14.   -3.    2.    5.06]
 [ 1.   -2.   13.    3.    6.88]
 [ 2.    2.    1.    9.    6.54]]
--------------------------------------------------------------------------------
Escalonamento
--------------------------------------------------------------------------------
Coluna 1 :
L2 <- L2 - 1 / 7 * L1
[[ 7.          1.         -2.          2.          2.38      ]
 [ 0.         13.85714286 -2.71428571  1.71428571  4.72      ]
 [ 1.         -2.         13.          3.          6.88      ]
 [ 2.          2.          1.          9.          6.54      ]]
L3 <- L3 - 1 / 7 * L1
[[ 7.          1.         -2.          2.          2.38      ]
 [ 0.         13.85714286 -2.71428571  1.71428571  4.72      ]
 [ 0.         -2.14285714 13.28571429  2.71428571  6.54      ]
 [ 2.          

g) $\begin{cases}
10x_1 +  x_2 +  x_3 + 2x_4 =  2.5 \\
  x_1 - 6x_2 - 2x_3 +  x_4 = -2.5 \\
      -  x_2 + 6x_3        =  2.1 \\
 2x_1 +  x_2 +  x_3 + 8x_4 =  0.1
\end{cases}$

In [37]:
mat_g = [
    [10, 1, 1, 2, 2.5],
    [1, -6, -2, 1, -2.5],
    [0, -1, 6, 0, 2.1],
    [2, 1, 1, 8, 0.1]
]
mat_g = np.array(mat_g)
resolve(mat_g)

--------------------------------------------------------------------------------
Matriz estendida
--------------------------------------------------------------------------------
[[10.   1.   1.   2.   2.5]
 [ 1.  -6.  -2.   1.  -2.5]
 [ 0.  -1.   6.   0.   2.1]
 [ 2.   1.   1.   8.   0.1]]
--------------------------------------------------------------------------------
Escalonamento
--------------------------------------------------------------------------------
Coluna 1 :
L2 <- L2 - 1 / 10 * L1
[[10.    1.    1.    2.    2.5 ]
 [ 0.   -6.1  -2.1   0.8  -2.75]
 [ 0.   -1.    6.    0.    2.1 ]
 [ 2.    1.    1.    8.    0.1 ]]
L4 <- L4 - 2 / 10 * L1
[[10.    1.    1.    2.    2.5 ]
 [ 0.   -6.1  -2.1   0.8  -2.75]
 [ 0.   -1.    6.    0.    2.1 ]
 [ 0.    0.8   0.8   7.6  -0.4 ]]
Coluna 2 :
L3 <- L3 - -1 / -6.1 * L2
[[10.          1.          1.          2.          2.5       ]
 [ 0.         -6.1        -2.1         0.8        -2.75      ]
 [ 0.          0.          6.3442623  -0.13114